# Feature Engineering


In [53]:
import pandas as pd
import numpy as np
from pathlib import Path
import glob

# Load the data (same as notebook 01)
data_path = Path("../data/tennis_atp-master")
match_files = sorted(glob.glob(str(data_path / "atp_matches_*.csv")))

# Exclude doubles files
match_files = [f for f in match_files if 'doubles' not in Path(f).name.lower()]

dfs = [pd.read_csv(f, low_memory=False) for f in match_files]
matches_df = pd.concat(dfs, ignore_index=True)

print(f"Loaded {len(matches_df):,} matches from {len(match_files)} files (doubles excluded)")


Loaded 941,649 matches from 139 files (doubles excluded)


In [54]:
def create_player_centric_dataset(df):
    """
    Transform match-centric data to player-centric data.
    Each match appears twice: once from winner's perspective, once from loser's.
    
    Returns DataFrame with:
    - player_* columns (the focal player's stats)
    - opponent_* columns (the opponent's stats)
    - delta_* columns (player - opponent differences)
    - win: binary label (1 = player won, 0 = player lost)
    """
    
    # Identify winner/loser stat columns with both prefix patterns:
    # - Full prefix: winner_, loser_
    # - Short prefix: w_, l_
    winner_cols_full = [c for c in df.columns if c.startswith('winner_')]
    loser_cols_full = [c for c in df.columns if c.startswith('loser_')]
    winner_cols_short = [c for c in df.columns if c.startswith('w_')]
    loser_cols_short = [c for c in df.columns if c.startswith('l_')]
    
    # Get the base stat names (without prefix)
    winner_stats = {c.replace('winner_', ''): c for c in winner_cols_full}
    loser_stats = {c.replace('loser_', ''): c for c in loser_cols_full}
    
    # Add short prefix columns
    winner_stats.update({c.replace('w_', ''): c for c in winner_cols_short})
    loser_stats.update({c.replace('l_', ''): c for c in loser_cols_short})
    
    # Find common stats that exist for both winner and loser
    common_stats = set(winner_stats.keys()) & set(loser_stats.keys())
    
    # Match-level columns (not player-specific)
    match_cols = [c for c in df.columns 
                  if not c.startswith('winner_') and not c.startswith('loser_')
                  and not c.startswith('w_') and not c.startswith('l_')]
    
    print(f"Match-level columns: {len(match_cols)}")
    print(f"Common player stats: {len(common_stats)}")
    
    # Create winner perspective (win = 1)
    winner_perspective = df[match_cols].copy()
    winner_perspective['win'] = 1
    
    for stat in common_stats:
        winner_perspective[f'player_{stat}'] = df[winner_stats[stat]]
        winner_perspective[f'opponent_{stat}'] = df[loser_stats[stat]]
    
    # Create loser perspective (win = 0)
    loser_perspective = df[match_cols].copy()
    loser_perspective['win'] = 0
    
    for stat in common_stats:
        loser_perspective[f'player_{stat}'] = df[loser_stats[stat]]
        loser_perspective[f'opponent_{stat}'] = df[winner_stats[stat]]
    
    # Combine both perspectives
    player_df = pd.concat([winner_perspective, loser_perspective], ignore_index=True)
    
    return player_df, list(common_stats)


player_df, stat_names = create_player_centric_dataset(matches_df)
print(f"\nPlayer-centric dataset shape: {player_df.shape}")
print(f"  - {player_df.shape[0]:,} rows (2x matches, one per player)")
print(f"  - {player_df.shape[1]} columns")


Match-level columns: 11
Common player stats: 19

Player-centric dataset shape: (1883298, 50)
  - 1,883,298 rows (2x matches, one per player)
  - 50 columns


In [55]:
# Compute delta features (player - opponent) for numeric stats
numeric_stats = []

# Stats to exclude from delta computation
exclude_stats = {'id'}

for stat in stat_names:
    if stat in exclude_stats:
        continue
        
    player_col = f'player_{stat}'
    opponent_col = f'opponent_{stat}'
    
    # Only compute deltas for numeric columns
    if pd.api.types.is_numeric_dtype(player_df[player_col]):
        delta_col = f'delta_{stat}'
        player_df[delta_col] = player_df[player_col] - player_df[opponent_col]
        numeric_stats.append(stat)

print(f"Created {len(numeric_stats)} delta features:")
for stat in sorted(numeric_stats):
    print(f"  - delta_{stat}")


Created 14 delta features:
  - delta_1stIn
  - delta_1stWon
  - delta_2ndWon
  - delta_SvGms
  - delta_ace
  - delta_age
  - delta_bpFaced
  - delta_bpSaved
  - delta_df
  - delta_ht
  - delta_rank
  - delta_rank_points
  - delta_seed
  - delta_svpt


In [56]:
# Verify the transformation - show sample of key columns
sample_cols = ['player_name', 'opponent_name', 'player_rank', 'opponent_rank', 
               'delta_rank', 'player_age', 'opponent_age', 'delta_age', 'win']

# Filter to columns that exist
available_cols = [c for c in sample_cols if c in player_df.columns]
player_df[available_cols].head(10)


,player_name,opponent_name,player_rank,opponent_rank,delta_rank,player_age,opponent_age,delta_age,win
0,Doug Smith,Peter Ledbetter,NaN,NaN,NaN,NaN,24.0,NaN,1
1,Louis Pretorius,Maurice Pollock,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Cecil Pedlow,John Mulvey,NaN,NaN,NaN,NaN,NaN,NaN,1
3,Tom Okker,Unknown Fearmon,NaN,NaN,NaN,24.3,NaN,NaN,1
4,Armistead Neely,Harry Sheridan,NaN,NaN,NaN,21.3,NaN,NaN,1
5,John Mcgrath,Brendan Kelly,NaN,NaN,NaN,NaN,NaN,NaN,1
6,Bob Howe,Kenneth Reid,NaN,NaN,NaN,42.9,NaN,NaN,1
7,Lew Hoad,Jim Buckley,NaN,NaN,NaN,33.6,31.2,2.4,1
8,Graydon Garner,Vivian Gotto,NaN,NaN,NaN,23.0,46.8,-23.8,1
9,Des Foley,Peter Mockler,NaN,NaN,NaN,NaN,NaN,NaN,1


In [57]:
# Verify win label distribution (should be 50/50)
print("Win label distribution:")
print(player_df['win'].value_counts())
print(f"\nWin rate: {player_df['win'].mean():.1%}")

print(f"\nFinal dataset shape: {player_df.shape}")
print(f"Columns by type:")
print(f"  - player_* columns: {len([c for c in player_df.columns if c.startswith('player_')])}")
print(f"  - opponent_* columns: {len([c for c in player_df.columns if c.startswith('opponent_')])}")
print(f"  - delta_* columns: {len([c for c in player_df.columns if c.startswith('delta_')])}")


Win label distribution:
win
1    941649
0    941649
Name: count, dtype: int64

Win rate: 50.0%

Final dataset shape: (1883298, 64)
Columns by type:
  - player_* columns: 19
  - opponent_* columns: 19
  - delta_* columns: 14


In [58]:
# Calculate percentage of rows with at least one missing delta feature (after 2005)

# Extract year from tourney_date (format: YYYYMMDD)
# Use pd.to_numeric with errors='coerce' to handle malformed values
player_df['year'] = pd.to_numeric(
    player_df['tourney_date'].astype(str).str[:4], 
    errors='coerce'
)

# Filter to matches after 2005
df_post_2005 = player_df[player_df['year'] > 2005]

# Get all delta columns
delta_cols = [c for c in player_df.columns if c.startswith('delta_')]

# Check for rows with at least one missing value in delta columns
rows_with_missing = df_post_2005[delta_cols].isna().any(axis=1).sum()
total_rows = len(df_post_2005)
pct_missing = (rows_with_missing / total_rows) * 100

print(f"Matches after 2005: {total_rows:,} rows")
print(f"Rows with at least one missing delta feature: {rows_with_missing:,}")
print(f"Percentage: {pct_missing:.2f}%")


Matches after 2005: 1,034,362 rows
Rows with at least one missing delta feature: 1,006,074
Percentage: 97.27%


In [59]:
# Calculate percentage of missing values for each delta feature (after 2005)
missing_pct = (df_post_2005[delta_cols].isna().sum() / len(df_post_2005) * 100).sort_values()

print("Missing values per delta feature (sorted least to most):")
print("-" * 50)
for col, pct in missing_pct.items():
    print(f"{col:25} {pct:6.2f}%")


Missing values per delta feature (sorted least to most):
--------------------------------------------------
delta_age                   0.55%
delta_rank                 14.67%
delta_rank_points          14.70%
delta_ht                   49.11%
delta_svpt                 67.68%
delta_ace                  67.68%
delta_1stIn                67.68%
delta_1stWon               67.68%
delta_bpSaved              67.68%
delta_df                   67.68%
delta_2ndWon               67.68%
delta_bpFaced              67.68%
delta_SvGms                67.68%
delta_seed                 91.96%


In [60]:
# Investigate why 67.68% of match stats are missing

# Pick one of the match stats to investigate (they should all have the same pattern)
df_post_2005['has_match_stats'] = df_post_2005['delta_ace'].notna()

# Check by year
print("Percentage of matches WITH detailed stats by year:")
print("-" * 50)
stats_by_year = df_post_2005.groupby('year')['has_match_stats'].mean() * 100
print(stats_by_year.to_string())

print("\n")

# Check by tournament level if the column exists
if 'tourney_level' in df_post_2005.columns:
    print("Percentage of matches WITH detailed stats by tournament level:")
    print("-" * 50)
    stats_by_level = df_post_2005.groupby('tourney_level')['has_match_stats'].mean() * 100
    print(stats_by_level.sort_values(ascending=False).to_string())


Percentage of matches WITH detailed stats by year:
--------------------------------------------------
year
2006    11.957729
2007    11.153432
2008    10.024752
2009    10.854071
2010    37.180681
2011    33.974606
2012    32.720969
2013    30.232637
2014    28.802243
2015    27.081460
2016    37.500000
2017    37.666503
2018    42.105961
2019    40.528288
2020    50.042845
2021    45.796129
2022    43.325695
2023    42.960831
2024    43.320383


Percentage of matches WITH detailed stats by tournament level:
--------------------------------------------------


C:\Users\shlok\AppData\Local\Temp\ipykernel_37520\2791250408.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_post_2005['has_match_stats'] = df_post_2005['delta_ace'].notna()


tourney_level
F     99.135447
O     95.312500
M     94.212736
A     90.381185
C     81.574974
G     79.582328
D     26.759443
15     0.000000
25     0.000000
S      0.000000


In [61]:
# Drop tournaments with little to no match statistics:
# - D (Davis Cup): Team competition with inconsistent stat recording
# - S (Satellites): Older lower-tier events, no stats
# - 15, 25 (ITF Futures): Lower professional tier, no detailed stats recorded

exclude_levels = ['D', 'S', '15', '25', 15, 25]  # Include both string and int versions
rows_before = len(df_post_2005)

df_filtered = df_post_2005[~df_post_2005['tourney_level'].isin(exclude_levels)].copy()

rows_after = len(df_filtered)
print(f"Rows before: {rows_before:,}")
print(f"Rows after:  {rows_after:,}")
print(f"Dropped:     {rows_before - rows_after:,} ({(rows_before - rows_after) / rows_before * 100:.1f}%)")
print(f"\nRemaining tournament levels: {sorted(df_filtered['tourney_level'].astype(str).unique())}")


Rows before: 1,034,362
Rows after:  392,984
Dropped:     641,378 (62.0%)

Remaining tournament levels: ['A', 'C', 'F', 'G', 'M', 'O']


In [62]:
# Add flag for Olympic tournaments
# Search by tournament name to capture historical Olympics (not just level 'O')
df_filtered['is_olympics'] = (
    (df_filtered['tourney_level'] == 'O') | 
    (df_filtered['tourney_name'].str.lower().str.contains('olympic', na=False))
)

print(f"Olympic matches: {df_filtered['is_olympics'].sum():,}")
print(f"Non-Olympic matches: {(~df_filtered['is_olympics']).sum():,}")


Olympic matches: 640
Non-Olympic matches: 392,344


In [63]:
# Olympic matches by year
olympic_matches = df_filtered[df_filtered['is_olympics']]
olympic_by_year = olympic_matches.groupby('year').size()

print("Olympic matches per year:")
print("-" * 30)
print(olympic_by_year.to_string())


Olympic matches per year:
------------------------------
year
2008    128
2012    128
2016    128
2021    128
2024    128


In [64]:
# Investigate historical Olympics - search by tournament name
olympic_keywords = ['olympic', 'olympics', 'jeux']

# Check if tourney_name column exists and search for Olympic-related tournaments
if 'tourney_name' in df_filtered.columns:
    mask = df_filtered['tourney_name'].str.lower().str.contains('|'.join(olympic_keywords), na=False)
    olympic_by_name = df_filtered[mask][['year', 'tourney_name', 'tourney_level']].drop_duplicates()
    
    print("Tournaments matching 'olympic' in name:")
    print("-" * 60)
    print(olympic_by_name.sort_values('year').to_string(index=False))


Tournaments matching 'olympic' in name:
------------------------------------------------------------
 year     tourney_name tourney_level
 2008 Beijing Olympics             A
 2012  London Olympics             A
 2016     Rio Olympics             A
 2021   Tokyo Olympics             A
 2024   Paris Olympics             O


In [70]:
# Drop delta_seed before calculating missingness
if 'delta_seed' in df_filtered.columns:
    df_filtered = df_filtered.drop(columns=['delta_seed'])
    print("Dropped 'delta_seed' column\n")

# Missingness summary for delta features only
delta_cols = [c for c in df_filtered.columns if c.startswith('delta_')]
print("\nMissingness summary (delta features only):")
print("-" * 50)
delta_missing = df_filtered[delta_cols].isnull().sum()
delta_missing_pct = (delta_missing / len(df_filtered) * 100).round(2)
delta_missing_summary = pd.DataFrame({
    'missing_count': delta_missing,
    'missing_pct': delta_missing_pct
})
delta_missing_summary = delta_missing_summary[delta_missing_summary['missing_count'] > 0].sort_values('missing_pct', ascending=False)
print(f"Delta features with missing values: {len(delta_missing_summary)}")
print(delta_missing_summary.to_string())

# Calculate percentage of records with at least one missing delta feature
rows_with_missing_delta = df_filtered[delta_cols].isna().any(axis=1).sum()
pct_records_missing_delta = (rows_with_missing_delta / len(df_filtered) * 100).round(2)
print(f"\nRecords with at least one missing delta feature: {rows_with_missing_delta:,} ({pct_records_missing_delta}%)")

print("\nMissingness summary (all columns):")
print("-" * 50)
missing = df_filtered.isnull().sum()
missing_pct = (missing / len(df_filtered) * 100).round(2)
missing_summary = pd.DataFrame({
    'missing_count': missing,
    'missing_pct': missing_pct
})
# Show only columns with missing values, sorted by percentage
missing_summary = missing_summary[missing_summary['missing_count'] > 0].sort_values('missing_pct', ascending=False)
print(f"Columns with missing values: {len(missing_summary)}")
print(missing_summary.to_string())



Missingness summary (delta features only):
--------------------------------------------------
Delta features with missing values: 13
                   missing_count  missing_pct
delta_svpt                 61368        15.62
delta_ace                  61368        15.62
delta_1stIn                61368        15.62
delta_1stWon               61368        15.62
delta_bpSaved              61368        15.62
delta_df                   61368        15.62
delta_SvGms                61372        15.62
delta_2ndWon               61368        15.62
delta_bpFaced              61368        15.62
delta_ht                   53776        13.68
delta_rank_points          12850         3.27
delta_rank                 12792         3.26
delta_age                    300         0.08

Records with at least one missing delta feature: 104,942 (26.7%)

Missingness summary (all columns):
--------------------------------------------------
Columns with missing values: 49
                      missing_count  

In [71]:
# Drop records with any missing delta features
delta_cols = [c for c in df_filtered.columns if c.startswith('delta_')]
rows_before = len(df_filtered)

# Keep only rows where all delta features are non-null
df_filtered = df_filtered[df_filtered[delta_cols].notna().all(axis=1)].copy()

rows_after = len(df_filtered)
rows_dropped = rows_before - rows_after

print(f"Rows before dropping missing delta features: {rows_before:,}")
print(f"Rows after dropping missing delta features:  {rows_after:,}")
print(f"Rows dropped: {rows_dropped:,} ({rows_dropped/rows_before*100:.2f}%)")

# Missingness summary after dropping
print("\nMissingness summary (after dropping records with missing delta features):")
print("-" * 50)
missing = df_filtered.isnull().sum()
missing_pct = (missing / len(df_filtered) * 100).round(2)
missing_summary = pd.DataFrame({
    'missing_count': missing,
    'missing_pct': missing_pct
})
# Show only columns with missing values, sorted by percentage
missing_summary = missing_summary[missing_summary['missing_count'] > 0].sort_values('missing_pct', ascending=False)
print(f"Columns with missing values: {len(missing_summary)}")
if len(missing_summary) > 0:
    print(missing_summary.to_string())
else:
    print("No missing values remaining!")


Rows before dropping missing delta features: 392,984
Rows after dropping missing delta features:  288,042
Rows dropped: 104,942 (26.70%)

Missingness summary (after dropping records with missing delta features):
--------------------------------------------------
Columns with missing values: 5
                missing_count  missing_pct
player_entry           233979        81.23
opponent_entry         233979        81.23
player_seed            178820        62.08
opponent_seed          178820        62.08
minutes                 10888         3.78


In [72]:
# Save the cleaned, filtered dataset
output_dir = Path("../data/clean")
output_dir.mkdir(parents=True, exist_ok=True)

output_path = output_dir / "atp_matches_cleaned.csv"
df_filtered.to_csv(output_path, index=False)

print(f"Saved to: {output_path}")
print(f"Final row count: {len(df_filtered):,}")


Saved to: ..\data\clean\atp_matches_cleaned.csv
Final row count: 288,042
